In [1]:
# # !pip install langchain langchain_nvidia_ai_endpoints faiss-cpu -q
# %pip install --upgrade --quiet  nest_asyncio

# # fixes a bug with asyncio and jupyter
# import nest_asyncio

# nest_asyncio.apply()

In [2]:
import os
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

In [3]:
import getpass

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

In [7]:
from langchain_community.document_loaders import WebBaseLoader

# Initialize WebBaseLoader with the specified URLs
loader = WebBaseLoader(["https://www.tribalscale.com/", "https://www.tribalscale.com/about", "https://www.tribalscale.com/careers"])

# Load documents from the web
documents = loader.load()

# Output the total number of documents loaded
print(len(documents))


3


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)

len(docs)
docs

[Document(page_content='Home | TribalScale1-416-800-0918Our ServicesProductTransformationDesignVenture StudioLLM CalculatorThought LeadershipBlogsPodcastsPress ReleaseCase StudiesAboutCareersConnect With UsOUR SERVICES ProductTransformationDesignVenture StudioLLM CalculatorTHOUGHT LEADERSHIP BlogsPodcastsPress ReleaseCOMPANY Case StudiesAboutCareersConnect With UsYour browser does not support HTML5 video.A mission toright the futureTribalScale is a global innovation firm that helps enterprises adapt and thrive in the digital era. Through Agile practices we transform teams, build best-in-class digital products, and create disruptive startups.75+ NUMBER OF PARTNERS500M+ UNIQUE USERS REACHED350+ PROJECTS SHIPPEDConnect With UsOur servicesA unique and comprehensive approach to digital innovationProductWe build and design cloud-native software solutions for web, mobile, and emerging tech.See MoreTransformationWe enable and empower enterprise teams on every aspect of Agile product developmen

In [11]:
print(docs[0].page_content)


Home | TribalScale1-416-800-0918Our ServicesProductTransformationDesignVenture StudioLLM CalculatorThought LeadershipBlogsPodcastsPress ReleaseCase StudiesAboutCareersConnect With UsOUR SERVICES ProductTransformationDesignVenture StudioLLM CalculatorTHOUGHT LEADERSHIP BlogsPodcastsPress ReleaseCOMPANY Case StudiesAboutCareersConnect With UsYour browser does not support HTML5 video.A mission toright the futureTribalScale is a global innovation firm that helps enterprises adapt and thrive in the digital era. Through Agile practices we transform teams, build best-in-class digital products, and create disruptive startups.75+ NUMBER OF PARTNERS500M+ UNIQUE USERS REACHED350+ PROJECTS SHIPPEDConnect With UsOur servicesA unique and comprehensive approach to digital innovationProductWe build and design cloud-native software solutions for web, mobile, and emerging tech.See MoreTransformationWe enable and empower enterprise teams on every aspect of Agile product development.See Moreour workOur


In [12]:
embeddings = NVIDIAEmbeddings()
query_result = embeddings.embed_query("Hello World!")
len(query_result)

1024

In [13]:
vector_store = FAISS.from_documents(documents=docs, embedding=NVIDIAEmbeddings())

In [20]:
vector_store.search("Who are you?", search_type="similarity", k=2)


[Document(page_content="About | TribalScale1-416-800-0918Our ServicesProductTransformationDesignVenture StudioLLM CalculatorThought LeadershipBlogsPodcastsPress ReleaseCase StudiesAboutCareersConnect With UsOUR SERVICES ProductTransformationDesignVenture StudioLLM CalculatorTHOUGHT LEADERSHIP BlogsPodcastsPress ReleaseCOMPANY Case StudiesAboutCareersConnect With UsA digital innovation firm with a mission to right the futureNot a massive development shop, VC firm, or design agency, but a unique group of skilled individuals, all feeding on one another's talent. Empowering businesses to grow their success; whether that's transforming their process, building products from a vision, establishing a brand, or supporting from the back-end.Our MissionRight The FutureVERBDefinition:To actively and purposefully shape a positive and transformative trajectory for the future. The term implies a commitment to innovation, resilience, and empowerment, with the goal of correcting, improving, or steering

In [25]:
llm = ChatNVIDIA(model="meta/llama2-70b")

chat = ChatNVIDIA(model="meta/llama2-70b", temperature=0.1, max_tokens=1000, top_p=1.0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)


doc_chain = load_qa_chain(chat , chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain(
    retriever=vector_store.as_retriever(),
    combine_docs_chain=doc_chain,
    memory=memory,
    question_generator=question_generator,
)

In [28]:
query = "What are the career options here?"
result = qa.invoke ({"question": query})
print(result.get("answer"))

  Based on the information provided, TribalScale offers various career options that align with their unique approach to digital innovation, empowerment, inclusivity, diversity, sustainability, and mission to "right the future." Some of the career options available at TribalScale include:

1. Product Transformation: TribalScale offers opportunities to work on transformative projects that redefine industries and empower businesses to chart new territories.
2. Design: As a digital innovation firm, TribalScale likely has a design team that focuses on creating innovative and inclusive designs for their clients.
3. Venture Studio: TribalScale's Venture Studio is dedicated to building and scaling startups that have the potential to create a positive impact on society.
4. LLM Calculator: TribalScale has developed an LLM Calculator, which is a tool that helps law firms and legal departments calculate their LLM (Master of Laws) return on investment.
5. Thought Leadership: TribalScale publishes b

In [29]:
query = "But why?"
result = qa({"question": query})
print(result.get("answer"))

  TribalScale prioritizes diversity and inclusion in their everyday actions, attends events, publishes blog series, and sets goals for their team because they believe in ensuring that all voices contribute, all values are understood, and everyone has a seat at the table. They practice Diversity and Inclusion in their daily actions and events to create a future that is both transformative and inclusive. By doing so, they aim to empower themselves, each other, and their clients to reach their goals and shape a future that embraces change, challenges the status quo, and promotes progress, inclusivity, and sustained success.


In [30]:
llm = ChatNVIDIA(model="llama2_70b", temperature=0.1, max_tokens=1000, top_p=1.0)

qa_prompt=QA_PROMPT

doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
)

c:\Users\acer\miniconda3\envs\resume\Lib\site-packages\langchain_nvidia_ai_endpoints\_statics.py:361: UserWarning: Model llama2_70b is deprecated. Using meta/llama2-70b instead.
  warnings.warn(


In [31]:
query = "What is Tribal Scale?"
result = qa({"question": query})
print(result.get("answer"))

  TribalScale's approach to digital innovation is unique and comprehensive, encompassing product development, transformation, design, venture studio, and thought leadership. Their mission to "right the future" means they embrace change, challenge the status quo, and work towards creating a future characterized by progress, inclusivity, and sustained success. TribalScale sees innovation as a guiding light leading them to solutions that redefine industries and empower businesses to chart new territories. Their approach goes beyond the conventional roles of a development shop, VC firm, or design agency, and they are committed to empowering businesses to grow their success. This approach aligns with their mission to shape a positive and transformative trajectory for the future, correcting, improving, or steering the course of what lies ahead.


In [32]:
query = "But why?"
result = qa({"question": query})
print(result.get("answer"))

  TribalScale prioritizes diversity and inclusion in their everyday actions, attends events, publishes blog series, and sets goals for their team because they believe in ensuring all voices contribute, all values are understood, and everyone has a seat at the table. They practice Diversity and Inclusion in everyday actions as well as attending events, publishing blog series, or setting goals for their team. This is evident in their mission to "right the future" by embracing change, challenging the status quo, and working towards creating a future characterized by progress, inclusivity, and sustained success. They envision a future where innovation isn't a luxury but a necessity, where businesses thrive by embracing change rather than fearing it. By prioritizing diversity and inclusion, TribalScale aims to create a future that is both transformative and inclusive.
